# TextBook. 파이썬을 이용한 웹크롤링과 스크레이핑

## 파이썬으로 시작하는 크롤링/스크레이핑 


> - 파이썬 기초 지식
- 웹 페이지 추출
- 웹 페이지에서 데이터 추출
- 데이터 저장
- 파이썬으로 스크레이핑하는 흐름

### 가상환경 사용법
> - 가상환경 생성 
- \> python -m venv 디렉토리명(myvenv) 
- \> .\scraping\Scripts\activate 
- (myvenv) > python --version 

> - 가상환경 종료 
- (myvenv) > .\scraping\Scripts\deactivate 
- \> dir/w

In [1]:
import os
from os.path import exists

def make_dir(dir_name):
    ''' 디렉토리 생성 '''
    
    if not exists(dir_name):
        os.mkdir(dir_name)
        msg = "{} 디렉토리를 생성하였습니다!".format(dir_name)
    else:
        msg = "{} 디렉토리가 이미 존재합니다!".format(dir_name)
        
    return msg

### 웹 페이지 추출
### c2-09_urlopen_encoding
> HTTP 헤더에서 인코딩 방식 추출

> - text/html
- text/html; charset=UTF-8
- text/html; charset=EUC-KR

cf. 연관함수 : HTTPMessage.info().get_content_charset()



In [2]:
import sys
from urllib.request import urlopen

In [3]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

# HTTP 헤더를 기반으로 인코딩 방식을 추출합니다
# (명시돼 있지 않을 경우 utf-8을 사용하게 합니다).
encoding = f.info().get_content_charset(failobj="utf-8")

# 인코딩 방식을 표준 오류에 출력합니다.
print('encoding:', encoding, file=sys.stderr)

encoding: utf-8


In [4]:
# 추출한 인코딩 방식으로 디코딩합니다.
text = f.read().decode(encoding)
# 웹 페이지의 내용을 표준 출력에 출력합니다.
text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<!-- Google Tag Manager -->\r\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-W9D5PM3\');</script>\r\n<!-- End Google Tag Manager -->\r\n<meta charset="utf-8"/>\r\n<title>한빛출판네트워크</title>\r\n<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛출판네트워크"/>\r\n<meta property="og:description" content="출판사, IT전문서, 대학교재, 경제경영, 어린이/유아,

### c2-10_urlopen_meta
> meta 태그에서 인코딩 방식 추출 
Content-Type 헤더의 값고 실제 사용되고 있는 인코딩 형식이 다를 수 있다. 
디코딩 처리때 UnicodeDecodeError가 발생 한다면 이러한 처리를 모방해서 구현하면 해결할 수 있다.

> - \< meta charset="utf-8" >
- \< meta http-equiv="Content-Type" content="text/html; charset="EUC-KR" >


cf. 연관함수 : HTTPMessage.info().get_content_charset()



In [5]:
import re
import sys
from urllib.request import urlopen

In [6]:
f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')
# bytes 자료형의 응답 본문을 일단 변수에 저장합니다.
bytes_content = f.read()  

type(bytes_content), bytes_content[:2**10]

(bytes,
 b'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<!-- Google Tag Manager -->\r\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-W9D5PM3\');</script>\r\n<!-- End Google Tag Manager -->\r\n<meta charset="utf-8"/>\r\n<title>\xed\x95\x9c\xeb\xb9\x9b\xec\xb6\x9c\xed\x8c\x90\xeb\x84\xa4\xed\x8a\xb8\xec\x9b\x8c\xed\x81\xac</title>\r\n<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title

In [7]:
# charset은 HTML의 앞부분에 적혀 있는 경우가 많으므로
# 응답 본문의 앞부분 1024바이트를 ASCII 문자로 디코딩해 둡니다.
# ASCII 범위 이위의 문자는 U+FFFD(REPLACEMENT CHARACTER)로 변환되어 예외가 발생하지 않습니다.
scanned_text = bytes_content[:2**10].decode('ascii', errors='replace')

type(scanned_text), scanned_text

(str,
 '<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<!-- Google Tag Manager -->\r\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-W9D5PM3\');</script>\r\n<!-- End Google Tag Manager -->\r\n<meta charset="utf-8"/>\r\n<title>������������������������</title>\r\n<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="������������������������"/>\r\n<meta property="og:description" c

In [8]:
scanned_text = bytes_content[:2**10].decode('utf-8', errors='replace')

type(scanned_text), scanned_text

(str,
 '<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<!-- Google Tag Manager -->\r\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\r\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\r\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\r\n\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\r\n})(window,document,\'script\',\'dataLayer\',\'GTM-W9D5PM3\');</script>\r\n<!-- End Google Tag Manager -->\r\n<meta charset="utf-8"/>\r\n<title>한빛출판네트워크</title>\r\n<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛출판네트워크"/>\r\n<meta property="og:description" content="출판사, IT전문서, 대학교재, 경제경영, 

In [9]:
# 디코딩한 문자열에서 정규 표현식으로 charset 값을 추출합니다.
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
if match:
    encoding = match.group(1)
else:
    # charset이 명시돼 있지 않으면 UTF-8을 사용합니다.
    encoding = 'utf-8'

# 추출한 인코딩을 표준 오류에 출력합니다.
print('encoding:', encoding, file=sys.stderr)

encoding: utf-8


In [10]:
# 추출한 인코딩으로 다시 디코딩합니다.
text = bytes_content.decode(encoding)

# 응답 본문을 표준 출력에 출력합니다.
print(text[:2**10])

<!DOCTYPE html>
<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-W9D5PM3');</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<title>한빛출판네트워크</title>
<link rel="shortcut icon" href="http://www.hanbit.co.kr/images/common/hanbit.ico"> 
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta property="og:type" content="website"/>
<meta property="og:title" content="한빛출판네트워크"/>
<meta property="og:description" content="출판사, IT전문서, 대학교재, 경제경영, 어린이/유아, MAKE, 실용/여행, 전자책, 인터넷 강의"/>
<meta property="og:image" content="http://www.hanbit.co

### 웹 페이지에서 데이터 추출
> - Regular Expression
- XML Parser

### c2-11_scrape_re
> 정규표현식으로 스크레이핑

In [11]:
! wget http://www.hanbit.co.kr/store/books/full_book_list.html

--2018-11-13 13:12:02--  http://www.hanbit.co.kr/store/books/full_book_list.html
Resolving www.hanbit.co.kr (www.hanbit.co.kr)... 218.38.58.195
Connecting to www.hanbit.co.kr (www.hanbit.co.kr)|218.38.58.195|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'full_book_list.html'

     0K .......... .......... .......... .......... .....      5.01M=0.009s

2018-11-13 13:12:02 (5.01 MB/s) - 'full_book_list.html' saved [46441]



In [12]:
! dir *.html

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src 디렉터리

2018-11-13  오후 01:12            46,441 full_book_list.html
               1개 파일              46,441 바이트
               0개 디렉터리  404,704,563,200 바이트 남음


In [13]:
# ! mkdir data
make_dir('data')

'data 디렉토리가 이미 존재합니다!'

In [14]:
! dir data

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src\data 디렉터리

2018-11-12  오전 09:53    <DIR>          .
2018-11-12  오전 09:53    <DIR>          ..
2018-10-29  오후 04:32    <DIR>          .ipynb_checkpoints
2018-11-07  오후 02:16             7,675 chicagomag_info.csv
2018-11-07  오후 03:45            10,404 chicagomag_info_ver2.csv
2018-10-29  오후 04:54               310 company.csv
2018-11-01  오후 04:26                62 girlgroup.json
2018-10-29  오후 04:33                27 hello.txt
2018-11-01  오후 04:29               162 member.json
2018-11-08  오후 07:24             4,201 movie_rank.csv
2018-11-09  오전 11:17               852 movie_top10.csv
2018-11-09  오전 11:20               852 movie_top10_v2.csv
2018-11-09  오전 09:21             2,007 musicrank_top100.csv
2018-11-09  오전 10:34             1,875 music_rank50.csv
2018-11-06  오후 03:20             1,140 opengov_2018_seoul_cctv_status_by_year.csv
2018-11-01  오후 04:31               193 person.json
2018-11-05  오후 04:59       

In [15]:
! ren full_book_list.html dp.html
! move dp.html data

        1개 파일을 이동했습니다.


In [16]:
! dir data\*.html

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src\data 디렉터리

2018-11-13  오후 01:12            46,441 dp.html
               1개 파일              46,441 바이트
               0개 디렉터리  404,704,563,200 바이트 남음


In [17]:
import re
from html import unescape

# 이전 절에서 다운로드한 파일을 열고 html이라는 변수에 저장합니다.
with open('./data/dp.html', encoding='utf-8') as f:
    html = f.read()

# re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출합니다.
print('check')
for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
    # 도서의 URL을 추출합니다.
    url = re.search(r'<a href="(.*?)">', partial_html).group(1)
    url = 'http://www.hanbit.co.kr' + url
    # 태그를 제거해서 도서의 제목을 추출합니다.
    title = re.sub(r'<.*?>', '', partial_html)
    title = unescape(title)
    print('url:', url)
    print('title:', title)
    print('---')

check
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B6994299591
title: 재미있고 빠른 한글 2권 : 기본 자음과 쌍자음
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B8463831992
title: 재미있고 빠른 한글 4권 : 복잡한 모음
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B3283906872
title: IT CookBook, 시스템 해킹과 보안(3판)
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B8608817158
title: IT 트렌드 스페셜 리포트 2019
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B4851649517
title: Accelerated C++
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B1266184916
title: 한 권으로 끝내는 딥러닝 텐서플로
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B8718279503
title: 파이썬으로 배우는 머신러닝의 교과서
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B5331040931
title: 좋은 사진을 만드는 ZAKO의 여행 사진 잘 찍는 법
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B3696148245
title: 2019 전기기사 필기 7개년 기출문제
---
url: http://www.hanbit.co.kr/store/books/look.php?p_code=B6727651495


### c2-12 : rss.xml
> RSS 파싱하기

> - 서울/경기도 지역의 날씨 받기
- http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109

In [18]:
! wget http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109

--2018-11-13 13:12:03--  http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109
Resolving www.weather.go.kr (www.weather.go.kr)... 182.161.120.118
Connecting to www.weather.go.kr (www.weather.go.kr)|182.161.120.118|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96129 (94K) [text/xml]
Saving to: 'mid-term-rss3.jsp@stnId=109'

     0K .......... .......... .......... .......... .......... 53% 2.96M 0s
    50K .......... .......... .......... .......... ...       100% 26.0M=0.02s

2018-11-13 13:12:03 (5.05 MB/s) - 'mid-term-rss3.jsp@stnId=109' saved [96129/96129]



In [19]:
! dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src 디렉터리

2018-11-13  오후 01:12    <DIR>          .
2018-11-13  오후 01:12    <DIR>          ..
2018-11-13  오전 11:34    <DIR>          .ipynb_checkpoints
2018-11-13  오후 01:12    <DIR>          data
2018-11-13  오전 09:38    <DIR>          download
2018-11-08  오후 01:37    <DIR>          driver
2018-11-07  오전 09:48    <DIR>          htmltest
2018-10-31  오후 03:10             8,947 IITP_C반_유선우-Copy1.ipynb
2018-10-31  오후 02:16             5,830 IITP_C반_유선우-Copy3.ipynb
2018-11-03  오후 04:06            11,860 IITP_C반_유선우.ipynb
2018-11-12  오전 11:32    <DIR>          images
2018-11-12  오전 10:41    <DIR>          img
2018-10-29  오후 01:13             6,084 Jupyter_Test.ipynb
2018-11-13  오후 01:12            96,129 mid-term-rss3.jsp@stnId=109
2018-10-30  오후 03:39    <DIR>          modules
2018-10-30  오후 03:17                92 my_first_module.py
2018-11-03  오후 06:01            33,308 naver_ranking_&_titanic-Copy2.ipynb
2018-11-13  오

In [20]:
! wget --help

GNU Wget 1.19.4, a non-interactive network retriever.
Usage: wget [OPTION]... [URL]...

Mandatory arguments to long options are mandatory for short options too.

Startup:
  -V,  --version                   display the version of Wget and exit
  -h,  --help                      print this help
  -b,  --background                go to background after startup
  -e,  --execute=COMMAND           execute a `.wgetrc'-style command

Logging and input file:
  -o,  --output-file=FILE          log messages to FILE
  -a,  --append-output=FILE        append messages to FILE
  -d,  --debug                     print lots of debugging information
  -q,  --quiet                     quiet (no output)
  -v,  --verbose                   be verbose (this is the default)
  -nv, --no-verbose                turn off verboseness, without being quiet
       --report-speed=TYPE         output bandwidth as TYPE.  TYPE can be bits
  -i,  --input-file=FILE           download URLs found in local or external FILE
  

In [21]:
# rss 데이터 저장
! wget rss.xml http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109 -O ./data/rss.xml

--2018-11-13 13:21:29--  http://rss.xml/
Resolving rss.xml (rss.xml)... failed: 요청한 이름이 올바르나, 요청한 종류의 데이터를 찾지 못했습니다. .
wget: unable to resolve host address 'rss.xml'
--2018-11-13 13:21:31--  http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109
Resolving www.weather.go.kr (www.weather.go.kr)... 121.78.77.90
Connecting to www.weather.go.kr (www.weather.go.kr)|121.78.77.90|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96129 (94K) [text/xml]
Saving to: './data/rss.xml'

     0K .......... .......... .......... .......... .......... 53% 6.35M 0s
    50K .......... .......... .......... .......... ...       100% 28.8M=0.009s

2018-11-13 13:21:31 (10.0 MB/s) - './data/rss.xml' saved [96129/96129]

FINISHED --2018-11-13 13:21:31--
Total wall clock time: 2.4s
Downloaded: 1 files, 94K in 0.009s (10.0 MB/s)


In [22]:
! dir data\*.xml

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src\data 디렉터리

2018-11-13  오후 01:21            96,129 rss.xml
               1개 파일              96,129 바이트
               0개 디렉터리  404,705,144,832 바이트 남음


### c2-13_scrape_rss
> ElementTree 모듈을 사용해 RSS 파싱

In [23]:
# ElementTree 모듈을 읽어 들입니다.
from xml.etree import ElementTree

# parse() 함수로 파일을 읽어 들이고 ElementTree 객체를 만듭니다.
tree = ElementTree.parse('./data/rss.xml')

# getroot() 메서드로 XML의 루트 요소를 추출합니다.
root = tree.getroot()


In [24]:
msg = " - tree : {} \n - root : {}".format(tree, root)
print(msg)

 - tree : <xml.etree.ElementTree.ElementTree object at 0x0000000005199F98> 
 - root : <Element 'rss' at 0x00000000051B5C78>


In [25]:
find_data = root.findall('channel/item/description/body/location/data')
len(find_data), find_data[0]

(455, <Element 'data' at 0x00000000051DF8B8>)

In [26]:
# findall() 메서드로 요소 목록을 추출합니다.
# 태그를 찾습니다(자세한 내용은 RSS를 열어 참고해주세요).
for item in root.findall('channel/item/description/body/location/data'):
    # find() 메서드로 요소를 찾고 text 속성으로 값을 추출합니다.
    tm_ef = item.find('tmEf').text
    tmn   = item.find('tmn').text
    tmx   = item.find('tmx').text
    wf    = item.find('wf').text
    print(tm_ef, tmn, tmx, wf) # 출력합니다.

2018-11-16 00:00 4 14 구름많음
2018-11-16 12:00 4 14 구름많음
2018-11-17 00:00 4 11 구름많음
2018-11-17 12:00 4 11 구름많음
2018-11-18 00:00 3 10 구름많음
2018-11-18 12:00 3 10 구름많음
2018-11-19 00:00 1 9 구름조금
2018-11-19 12:00 1 9 구름조금
2018-11-20 00:00 0 9 구름조금
2018-11-20 12:00 0 9 구름조금
2018-11-21 00:00 1 10 구름많음
2018-11-22 00:00 0 9 구름조금
2018-11-23 00:00 1 9 구름조금
2018-11-16 00:00 6 13 구름많음
2018-11-16 12:00 6 13 구름많음
2018-11-17 00:00 5 11 구름많음
2018-11-17 12:00 5 11 구름많음
2018-11-18 00:00 3 10 구름많음
2018-11-18 12:00 3 10 구름많음
2018-11-19 00:00 3 8 구름조금
2018-11-19 12:00 3 8 구름조금
2018-11-20 00:00 1 8 구름조금
2018-11-20 12:00 1 8 구름조금
2018-11-21 00:00 2 8 구름많음
2018-11-22 00:00 2 8 구름조금
2018-11-23 00:00 3 9 구름조금
2018-11-16 00:00 3 14 구름많음
2018-11-16 12:00 3 14 구름많음
2018-11-17 00:00 3 11 구름많음
2018-11-17 12:00 3 11 구름많음
2018-11-18 00:00 2 11 구름많음
2018-11-18 12:00 2 11 구름많음
2018-11-19 00:00 0 9 구름조금
2018-11-19 12:00 0 9 구름조금
2018-11-20 00:00 0 9 구름조금
2018-11-20 12:00 0 9 구름조금
2018-11-21 00:00 1 10 구름많음
2018-11-22 00:00 0

2018-11-17 00:00 1 10 구름많음
2018-11-17 12:00 1 10 구름많음
2018-11-18 00:00 0 11 구름많음
2018-11-18 12:00 0 11 구름많음
2018-11-19 00:00 1 10 구름조금
2018-11-19 12:00 1 10 구름조금
2018-11-20 00:00 -1 9 구름조금
2018-11-20 12:00 -1 9 구름조금
2018-11-21 00:00 0 11 구름많음
2018-11-22 00:00 0 11 구름조금
2018-11-23 00:00 1 11 구름조금
2018-11-16 00:00 3 13 구름많음
2018-11-16 12:00 3 13 구름많음
2018-11-17 00:00 2 11 구름많음
2018-11-17 12:00 2 11 구름많음
2018-11-18 00:00 2 11 구름많음
2018-11-18 12:00 2 11 구름많음
2018-11-19 00:00 0 9 구름조금
2018-11-19 12:00 0 9 구름조금
2018-11-20 00:00 -1 8 구름조금
2018-11-20 12:00 -1 8 구름조금
2018-11-21 00:00 0 10 구름많음
2018-11-22 00:00 -1 9 구름조금
2018-11-23 00:00 1 10 구름조금
2018-11-16 00:00 4 14 구름많음
2018-11-16 12:00 4 14 구름많음
2018-11-17 00:00 3 10 구름많음
2018-11-17 12:00 3 10 구름많음
2018-11-18 00:00 2 11 구름많음
2018-11-18 12:00 2 11 구름많음
2018-11-19 00:00 3 11 구름조금
2018-11-19 12:00 3 11 구름조금
2018-11-20 00:00 0 10 구름조금
2018-11-20 12:00 0 10 구름조금
2018-11-21 00:00 0 11 구름많음
2018-11-22 00:00 2 12 구름조금
2018-11-23 00:00 2 11 구름조금
201

## 데이터 저장
> - TEXT파일로 저장 : CSV/TSV 형식, JSON 형식
- DataBase에 저장 : SQLite3, MySQL, Oracle, MongoDB

### c2-14_save_csv_join
> CSV(Comma-Seperated Values) 형식으로 저장

> - str.join() 메소드 사용
- (venb) $ python OOO.py > OOO.csv

In [27]:
# 첫 번째 줄에 헤더를 작성합니다.
print('rank,city,population')  

rank,city,population


In [28]:
# join() 메서드의 매개변수로 전달한 list는 str이어야 하므로 주의해 주세요.
print(','.join(['1', '상하이', '24150000']))
print(','.join(['2', '카라치', '23500000']))
print(','.join(['3', '베이징', '21516000']))
print(','.join(['4', '텐진', '14722100']))
print(','.join(['5', '이스탄불', '14160467']))

1,상하이,24150000
2,카라치,23500000
3,베이징,21516000
4,텐진,14722100
5,이스탄불,14160467


### c2-15_save_csv
> CSV(Comma-Seperated Values) 형식으로 저장

In [29]:
import csv

# 파일을 엽니다. newline=''으로 줄바꿈 코드의 자동 변환을 제어합니다.
csv_file = './data/top_cities_1.csv'

with open(csv_file, 'w', newline='') as f:
    # csv.writer는 파일 객체를 매개변수로 지정합니다.
    writer = csv.writer(f)  
    # 첫 번째 줄에는 헤더를 작성합니다.
    writer.writerow(['rank', 'city', 'population'])  
    # writerows()에 리스트를 전달하면 여러 개의 값을 출력합니다.
    writer.writerows([
        [1, '상하이', 24150000],
        [2, '카라치', 23500000],
        [3, '베이징', 21516000],
        [4, '텐진', 14722100],
        [5, '이스탄불', 14160467],
    ])

In [30]:
# 확인
with open(csv_file, 'r') as f:
    data = f.read()

print(data)

rank,city,population
1,상하이,24150000
2,카라치,23500000
3,베이징,21516000
4,텐진,14722100
5,이스탄불,14160467



### c2-16_save_csv_dict
> 딕셔너리 형식으로 저장

In [31]:
import csv

csv_file = './data/top_cities_2.csv'
with open(csv_file, 'w', newline='') as f:
    # 첫 번째 매개변수에 파일 객체
    # 두 번째 매개변수에 필드 이름 리스트를 지정합니다.
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
      # 첫 번째 줄에 헤더를 입력합니다.
    writer.writeheader()
    # writerows()로 여러 개의 데이터를 딕셔너리 형태로 작성합니다.
    writer.writerows([
        {'rank': 1,  'city': '상하이',   'population': 24150000},
        {'rank': 2,  'city': '카라치',   'population': 23500000},
        {'rank': 3,  'city': '베이징',   'population': 21516000},
        {'rank': 4,  'city': '텐진',     'population': 14722100},
        {'rank': 5,  'city': '이스탄불', 'population': 14160467},
    ])

In [32]:
# 확인
with open(csv_file, 'r') as f:
    data = f.readlines()
    
for idx in range(len(data)):    
    print(data[idx], end='')
    if idx==0:
        print('-'*20)
    

rank,city,population
--------------------
1,상하이,24150000
2,카라치,23500000
3,베이징,21516000
4,텐진,14722100
5,이스탄불,14160467


### c2-17_save_json
> JSON 형식으로 저장

In [33]:
import json

cities = [
    {'rank': 1,  'city': '상하이',   'population': 24150000},
    {'rank': 2,  'city': '카라치',   'population': 23500000},
    {'rank': 3,  'city': '베이징',   'population': 21516000},
    {'rank': 4,  'city': '텐진',     'population': 14722100},
    {'rank': 5,  'city': '이스탄불', 'population': 14160467},
]

print(json.dumps(cities))

[{"rank": 1, "city": "\uc0c1\ud558\uc774", "population": 24150000}, {"rank": 2, "city": "\uce74\ub77c\uce58", "population": 23500000}, {"rank": 3, "city": "\ubca0\uc774\uc9d5", "population": 21516000}, {"rank": 4, "city": "\ud150\uc9c4", "population": 14722100}, {"rank": 5, "city": "\uc774\uc2a4\ud0c4\ubd88", "population": 14160467}]


In [36]:
print(json.dumps(cities, ensure_ascii=False, indent=4))

# ensure_ascii 값을 True로 하면 한글로 된 글자들을 ascii코드로 변환해서 저장하게 된다.

[
    {
        "rank": 1,
        "city": "상하이",
        "population": 24150000
    },
    {
        "rank": 2,
        "city": "카라치",
        "population": 23500000
    },
    {
        "rank": 3,
        "city": "베이징",
        "population": 21516000
    },
    {
        "rank": 4,
        "city": "텐진",
        "population": 14722100
    },
    {
        "rank": 5,
        "city": "이스탄불",
        "population": 14160467
    }
]


### c2-18_save_sqlite3
> 데이터베이스(SQLite3)에 저장

> - SQLite3 는 파일기반의 간단한 관계형 데이터베이스
- SQL 구문을 사용해 데이터를 읽고 쓰기

In [38]:
# ! mkdir database
make_dir('database')


# 맨 윗부분 dir 생성 관련 함수를 참고해야 하는 부분이다.
# 원래라면 ! mkdir 명령어를 써야하지만 위에 함수를 통해서 make_dir 함수로 생성하게 된 것이다.

'database 디렉토리를 생성하였습니다!'

In [39]:
import sqlite3

# top_cities.db 파일을 열고 연결을 변수에 저장합니다.
db_path = './database/top_cities.db'
conn = sqlite3.connect(db_path)

# 커서를 추출합니다.
c = conn.cursor()

# execute() 메서드로 SQL 구문을 실행합니다.
# 스크립트를 여러 번 사용해도 같은 결과를 출력할 수 있게 cities 테이블이 존재하는 경우 제거합니다.
c.execute('DROP TABLE IF EXISTS cities')

# cities 테이블을 생성합니다.
c.execute('''
    CREATE TABLE cities (
        rank integer,
        city text,
        population integer
    )
''')

In [40]:
# execute() 메서드의 두 번째 매개변수에는 파라미터를 지정할 수 있습니다.
# SQL 내부에서 파라미터로 변경할 부분(플레이스홀더)은 ?로 지정합니다.
c.execute('INSERT INTO cities VALUES (?, ?, ?)', (1, '상하이', 24150000))

In [41]:
# 파라미터가 딕셔너리일 때는 플레이스홀더를 :<이름> 형태로 지정합니다.
c.execute('INSERT INTO cities VALUES (:rank, :city, :population)',
          {'rank': 2, 'city': '카라치', 'population': 23500000}) 

In [42]:
# executemany() 메서드를 사용하면 여러 개의 파라미터를 리스트로 지정해서
# 여러 개(현재 예제에서는 3개)의 SQL 구문을 실행할 수 있습니다.
c.executemany('INSERT INTO cities VALUES (:rank, :city, :population)', [
    {'rank': 3, 'city': '베이징', 'population': 21516000},
    {'rank': 4, 'city': '텐진', 'population': 14722100},
    {'rank': 5, 'city': '이스탄불', 'population': 14160467},
])

In [43]:
# 변경사항을 커밋(저장)합니다.
conn.commit()

In [44]:
# 저장한 데이터를 추출합니다.
c.execute('SELECT * FROM cities')

# 쿼리의 결과는 fetchall() 메서드로 추출할 수 있습니다.
for row in c.fetchall():
    # 추출한 데이터를 출력합니다.
    print(row)

# 연결을 닫습니다.
conn.close()

(1, '상하이', 24150000)
(2, '카라치', 23500000)
(3, '베이징', 21516000)
(4, '텐진', 14722100)
(5, '이스탄불', 14160467)


In [47]:
# 데이터를 데이타베이스로 만들어서 저장해 놓으면 나중을 위해서도 좋다.
# 데이터베이스를 만들 때는 해당하는 값들을 잘 매치해서 만들어 주는 것이다.

### c2-19_python_scraper
> 파이썬으로 스크레이핑하는 프로세스

> - 웹 페이지를 추출
- 스크레이핑
- 데이터를 저장

### main( ) 함수에서 차례대로 호출
- fetch(url) : 매개변수로 url을 받고 지정한 URL의 웹 페이지를 추출
- scrape(html) : 매개변수로 html을 받고, 정규표현식을 사용해 HTML에서 도서 정보를 추출
- save(db_path, books) : 매개변수로 books라는 도서 목록을 받고, SQLite 데이터베이스에 저장

In [49]:
# ! mkdir modules
make_dir('modules')

'modules 디렉토리가 이미 존재합니다!'

In [50]:
%%writefile  ./modules/python_scraper.py 

import re
import sqlite3
from urllib.request import urlopen
from html import unescape

def main():
    """
    메인 처리입니다.
    fetch(), scrape(), save() 함수를 호출합니다.
    """
    
    url = 'http://www.hanbit.co.kr/store/books/full_book_list.html'
    db_path = './database/books.db'
    
    html = fetch(url)
    books = scrape(html)
    save(db_path, books)


def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출합니다.
    웹 페이지의 인코딩 형식은 Content-Type 헤더를 기반으로 알아냅니다.
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식을 추출합니다.
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열을 디코딩합니다.
    html = f.read().decode(encoding)
    return html


def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출합니다.
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출합니다.
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목을 추출합니다.
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append({'url': url, 'title': title})
    
    return books


def save(db_path, books):
    """
    매개변수 books로 전달된 도서 목록을 SQLite 데이터베이스에 저장합니다.
    데이터베이스의 경로는 매개변수 dp_path로 지정합니다.
    반환값: None(없음)
    """
    # 데이터베이스를 열고 연결을 확립합니다.
    conn = sqlite3.connect(db_path)
    
    # 커서를 추출합니다.
    c = conn.cursor()
    
    # execute() 메서드로 SQL을 실행합니다.
    # 스크립트를 여러 번 실행할 수 있으므로 기존의 books 테이블을 제거합니다.
    c.execute('DROP TABLE IF EXISTS books')
    
    # books 테이블을 생성합니다.
    c.execute('''
        CREATE TABLE books (
            title text,
            url text
        )
    ''')
    
    # executemany() 메서드를 사용하면 매개변수로 리스트를 지정할 수 있습니다.
    c.executemany('INSERT INTO books VALUES (:title, :url)', books)
    
    # 변경사항을 커밋(저장)합니다.
    conn.commit()
    
    # 연결을 종료합니다.
    conn.close()

# python 명령어로 실행한 경우 main() 함수를 호출합니다.
# 이는 모듈로써 다른 파일에서 읽어 들였을 때 main() 함수가 호출되지 않게 하는 것입니다.
# 파이썬 프로그램의 일반적인 작성 방식입니다.
if __name__ == '__main__':
    main()
    

Writing ./modules/python_scraper.py


In [51]:
! dir/w modules\*.py

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src\modules 디렉터리

my_area.py           my_module.py         my_module1.py
my_module2.py        my_module_test1.py   my_module_test2.py
my_module_test3.py   python_scraper.py    
               8개 파일               4,222 바이트
               0개 디렉터리  404,703,809,536 바이트 남음


In [53]:
# 파이썬 실행1
% run modules/python_scraper.py 

In [54]:
# # 파이썬 실행2
! python modules/python_scraper.py 

In [55]:
! dir database

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3EFF-82E2

 C:\Users\student\Anaconda_src\database 디렉터리

2018-11-13  오후 02:05    <DIR>          .
2018-11-13  오후 02:05    <DIR>          ..
2018-11-13  오후 02:05            16,384 books.db
2018-11-13  오후 01:43             8,192 top_cities.db
               2개 파일              24,576 바이트
               2개 디렉터리  404,703,870,976 바이트 남음


In [56]:
# ! sqlite3 ./database/books.db

## 중요점!!!
### 메인 함수에서 다른 함수들을 불러오는 모듈화하는 과정을 잘 이해해서 활용해야 한다!!!!
### 클래스와 같은 의미인 것임. 각각의 단계들을 모듈화해서 정리해 놓으면 엄청 편리하다는 것을 알아야 한다!!!
